# Applied Data Science Capstone by IBM on Coursera

## Best neighborhoods for millennials to live in Toronto

## Introduction: Business Problem

Toronto, as the largest metropolitan areas in Canada, is very diverse and is the financial captials of Canada. The exciting employment opportunities have attracted many Canadians and immigrants to migrate to Toronto, especially the millennials of age 22 and 37 years old. According to UBS Global Real Estate Bubble Index in 2019, Toronto is ranked number 2 for the the greatest risk of a housing bubble and being the most overpriced cities in the world.

Because of the steeply inflating housing markets, many millennials are not able to afford living in Toronto. They are forced to live in the suburbs of Ontario and spend many hours commuting to the city for work. According to Statistics Canada, the millennial cohort had a median after-tax household income of $44,093 in 2016.   

In this project, I am interested in investigating whether or not there are still neighbourhoods suitable and affordable for millennials to live in Toronto. A good old rule of thumb is that we shouldn't be spending more than 30% of our income on rent. Therefore, I will assess whether neighbourhoods with average rent less than $13230 a year or $1102 a month. In this project, I will focus on millenials that are only looking for a bachelor's unit. In addition to rent, I will also utilize Foursquare API to determine the first 100 venues closest to each neighbourhood. I will perform K-Means clustering to cluster them based on categories. Lastly, I will conclude the ideal neighbourhoods by taking rent, infrastructure around the neighbourhood, such as TTC stations, hospitals, police stations, and restaurants into account.

## Data

Aspects that will be taken into account in this problem are:
* Rent of Bachelor's unit in each neighbourhood
* Number of Venues in the neighbourhood
* Type of Venues (Infrastructure) in the neighbourhood
* Distance from each neighbourhood to downtown Toronto

Below is a list of data I will be using to investigate this problem:
* Toronto neighbourhood data that was already processed in Week 3 as the center of each neighbourhood. 
* The average rent of a Bachelor's by neighbourhood data found in this link: https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/Table?TableId=2.2.11&GeographyId=2270&GeographyTypeId=3&DisplayAs=Table&GeograghyName=Toronto#Row%20/%20Apartment
* Number of Venues and categories in each neighbourhood using **FourSquare API**.

## Methodology

### Exploratory Data Analysis

## Results

## Discussion

## Conclusion